<a href="https://colab.research.google.com/github/hallpaz/drl/blob/main/notebooks/policy_reinforce.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

O objetivo deste projeto é implementar um ambiente de simulação no framework Gym para resolver um problema específico. Neste problema, há uma coluna denominada "Random" contendo 50 posições, cada uma com um valor aleatório entre 1 e 100 (distribuição uniforme). Além disso, há um campo chamado "Target" que recebe um valor aleatório entre 1 e 100 (distribuição uniforme).

A missão do agente é selecionar até 5 posições da coluna "Random", uma de cada vez, de modo que a soma dos valores dessas 5 posições alcance o valor do campo "Target", indicando o fim do episódio.

In [ ]:
!pip install gymnasium

É importante destacar que o agente só pode "olhar" para uma posição de cada vez da coluna "Random".

Um episódio é finalizado se a soma de target foi satisfeita ou todas os valores na coluna Random foram vistos ao menos 1 vez.

Implementação:
1 - Criação do Ambiente de simulação utilizando o framework Gym.
2 - Definição inicial dos hiperparâmetros.
3 - Arquitetura da Rede Neural com Tensorflow.
4 - Treinamento do Agente.
5 - Teste e Demonstração do Agente Maduro.

In [ ]:
import gymnasium as gym
import numpy as np
import tensorflow as tf
import tensorflow_probability as tfp

In [ ]:
COMBINE = 1
DONT_COMBINE = 0

In [ ]:
class MatchingEnv(gym.Env):
  def __init__(self, options_size=50, target_size=1, options_limit=5):
    self.options_size = options_size
    self.target_size = target_size
    self.options_limit = options_limit
    self.action_space = [0, 1]

    self.reset()

  def reset(self):
    self.options = np.random.randint(1, 101, self.options_size)
    self.target = np.random.randint(1, 101, self.target_size)
    self.current_option_index = 0
    self.current_target_index = 0
    self.selected = []

    return (self.options[self.current_option_index],
            self.target[self.current_target_index],
            self.options_size - 1,
            len(self.selected))

  def step(self, action):
    done = False
    reward = 0
    if action == COMBINE:
      current_value = self.options[self.current_option_index]
      self.selected.append(current_value)
      # avança para o próximo valor
      self.current_option_index += 1
      # calcula o quanto falta em relação ao target
      # self.target[self.current_target_index] -= current_value
      remaining_value = self.target[self.current_target_index] - sum(self.selected)
      # quantos passos ainda faltam
      remaining_steps = self.options_size - self.current_option_index - 1

      if remaining_value < 0:
        reward = -sum(self.selected)
      else:
        reward = current_value

      # Falta recompensa
    else:
      self.current_option_index += 1
      # calcula o quanto falta em relação ao target
      remaining_value = self.target[self.current_target_index] - sum(self.selected)
      # quantos passos ainda faltam
      remaining_steps = self.options_size - self.current_option_index - 1

    if remaining_value == 0:
      reward = 100 * 10**(self.options_limit - len(self.selected))
      done = True

    if remaining_steps < 0 and not self.selected:
      reward = -100

    if remaining_steps < 0 or len(self.selected) >= self.options_limit or reward < 0:
      done = True

    next_state = (self.options[self.current_option_index % self.options_size],
                remaining_value,
                remaining_steps,
                len(self.selected))

    return next_state, reward, done, {}

In [ ]:
env = MatchingEnv()

A cada seleção que o agente aposta em fazer, as posições da coluna "Random" são sorteadas novamente, exceto aquelas que foram previamente escolhidas pelo agente.

In [ ]:
class Model(tf.keras.Model):
  def __init__(self, out_size):
    super().__init__()
    self.d1 = tf.keras.layers.Dense(30, activation='relu')
    self.d2 = tf.keras.layers.Dense(30, activation='relu')
    self.out = tf.keras.layers.Dense(out_size, activation='softmax')

  def call(self, input_data):
    x = tf.convert_to_tensor(input_data)
    x = self.d1(x)
    x = self.d2(x)
    x = self.out(x)
    return x

In [ ]:
class Agent():
  def __init__(self):
    self.model = Model(2)
    self.opt = tf.keras.optimizers.Adam(learning_rate=0.001)
    self.gamma = 1

  def act(self, state):
    prob = self.model(np.array([state]))
    dist = tfp.distributions.Categorical(probs=prob, dtype=tf.float32)
    action = dist.sample()
    return int(action.numpy()[0])

  def a_loss(self,prob, action, reward):
    dist = tfp.distributions.Categorical(probs=prob, dtype=tf.float32)
    log_prob = dist.log_prob(action)
    loss = -log_prob*reward
    return loss

  def train(self, states, rewards, actions):
    sum_reward = 0
    discnt_rewards = []
    rewards.reverse()
    for r in rewards:
      sum_reward = r + self.gamma*sum_reward
      discnt_rewards.append(sum_reward)
      discnt_rewards.reverse()

    for state, reward, action in zip(states, discnt_rewards, actions):
      with tf.GradientTape() as tape:
        p = self.model(np.array([state]), training=True)
        loss = self.a_loss(p, action, reward)
        grads = tape.gradient(loss, self.model.trainable_variables)
        self.opt.apply_gradients(zip(grads, self.model.trainable_variables))

In [ ]:
agent = Agent()
steps = 5000

In [ ]:
for s in range(steps):
  done = False
  state = env.reset()
  total_reward = 0
  rewards = []
  states = []
  actions = []
  while not done:
    action = agent.act(state)
    next_state, reward, done, _ = env.step(action)
    rewards.append(reward)
    states.append(state)
    actions.append(action)
    state = next_state
    total_reward += reward

    if done:
      agent.train(states, rewards, actions)
      print(env.target, env.selected)
      print("total reward after {} steps is {}".format(s, total_reward))

A saída de streaming foi truncada nas últimas 5000 linhas.
[76] [34, 58]
total reward after 47500 steps is -58
[81] [25, 40, 28]
total reward after 47501 steps is -28
[72] [72]
total reward after 47502 steps is 1000000
[100] [31, 47, 31]
total reward after 47503 steps is -31
[69] [97]
total reward after 47504 steps is -97
[91] [41, 77]
total reward after 47505 steps is -77
[42] [99]
total reward after 47506 steps is -99
[99] [65, 31, 90]
total reward after 47507 steps is -90
[2] [6]
total reward after 47508 steps is -6
[88] [38, 75]
total reward after 47509 steps is -75
[28] [43]
total reward after 47510 steps is -43
[93] [83, 46]
total reward after 47511 steps is -46
[15] [20]
total reward after 47512 steps is -20
[25] [11, 4, 10]
total reward after 47513 steps is 10015
[83] [90]
total reward after 47514 steps is -90
[28] [75]
total reward after 47515 steps is -75
[15] [28]
total reward after 47516 steps is -28
[65] [16, 69]
total reward after 47517 steps is -69
[36] [94]
total reward